# Hackathon FinTech Case Itmo
## Kirill Zakharov
2022

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
from skmultilearn.model_selection import iterative_train_test_split
import os
import json
import sklearn.utils as sku
from sklearn.metrics import fbeta_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

# plt.style.use('ggplot')
# sns.set_palette('mako')
sns.set_style('darkgrid')

In [2]:
# !pip install xgboost
from xgboost import XGBClassifier

In [3]:
PAYMENTS_TRAIN_PATH = 'data/data/payments_train.csv'
TARGET_TRAIN_PATH = 'data/data/target_train.csv'
PAYMENTS_TEST_PATH = 'data/data/payments_test.csv'
CLIENT_ID_TEST_PATH = 'data/data/client_id_test.csv'

In [4]:
payments_dtypes = {
    'client_id': str,
    'contractor_id': str,
    'is_outgoing': bool,
    'amount': 'uint64',
    'dt_day': 'uint16',
    'dt_hour': 'uint8',
    'channel': pd.CategoricalDtype()
}
for i in range(12):
    payments_dtypes[f'flag_{i}'] = bool

In [5]:
payments = pd.read_csv(PAYMENTS_TRAIN_PATH, dtype=payments_dtypes)

In [22]:
payments.head()

,client_id,contractor_id,is_outgoing,amount,dt_day,dt_hour,channel,flag_0,flag_1,flag_2,flag_3,flag_4,flag_5,flag_6,flag_7,flag_8,flag_9,flag_10,flag_11
0,569703,NaN,True,8674442,56,12,app,False,False,False,False,False,False,False,False,False,False,False,False
1,696595,3920,True,5714350,311,19,web,False,False,False,False,False,False,False,False,False,False,False,False
2,368467,NaN,True,3720501,175,13,NaN,False,False,False,True,False,False,False,False,False,False,False,False
3,421133,NaN,True,311542,68,14,NaN,False,False,False,True,False,False,False,False,False,False,False,False
4,365044,24686,True,705918747,171,15,app,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25937985,532940,NaN,True,904019,122,1,pos,True,False,False,False,False,False,False,False,False,False,False,False
25937986,923040,NaN,False,27261705,94,12,NaN,False,False,False,False,False,False,False,False,False,False,False,True
25937987,807703,58960,False,18409310,119,13,NaN,False,False,False,False,False,False,False,False,False,False,False,True
25937988,890561,NaN,True,7132,213,8,NaN,False,False,False,True,False,False,False,False,False,False,False,False


In [6]:
sort_payments = payments.sort_values(by=['client_id', 'dt_day', 'dt_hour'])

In [7]:
sort_payments['contractor_id'] = sort_payments['contractor_id'].fillna(method='bfill')

In [8]:
payments = sort_payments

In [10]:
payments['contractor_id'].value_counts()

471487    493980
58960     254572
602806    102655
919849     50886
964772     48490
           ...  
919446         1
486787         1
447516         1
103910         1
292976         1
Name: contractor_id, Length: 811473, dtype: int64

In [9]:
payments['cos'] = np.cos(payments['dt_day'] * np.pi / 180)
payments['cos_h'] = np.cos(payments['dt_hour']) #удалить
payments['sin'] = np.sin(payments['dt_day'] * np.pi / 180)
payments['sin_h'] = np.sin(payments['dt_hour'])  #удалить

In [6]:
contractors = payments['contractor_id']

In [72]:
target_dtypes = {
    'client_id': str
}
for i in range(35):
    target_dtypes[f'type_{i}'] = int

In [73]:
target = pd.read_csv(TARGET_TRAIN_PATH, dtype=target_dtypes)
target.head()

,client_id,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
0,775943,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,992314,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,255821,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,1,0
3,188791,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
4,46092,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# target['client_id'].value_counts()

In [ ]:
# target.shape

In [ ]:
payments['client_id'].value_counts() # данные о транзакциях есть по всем клиентам

In [74]:
target = target.set_index('client_id')

## Feature Engineering

In [33]:
def percentage_outgoing(x):
    counts = x.value_counts(normalize=True)
    if len(counts.index) == 2:
        return counts[0]
    
    elif len(counts.index) == 1 and counts.index == 0:
        return 1
    
    elif len(counts.index) == 1 and counts.index == 1:
        return 0
    
    else:
        return 'Error'

In [24]:
from scipy.stats import skew, kurtosis

In [25]:
def quantile_25(x):
    return x.quantile(.25)

def quantile_50(x):
    return x.quantile(.5)

def quantile_75(x):
    return x.quantile(.75)

In [36]:
# # dask

# import dask.dataframe as dd
# payments_dd = dd.read_csv(PAYMENTS_TRAIN_PATH, dtype=payments_dtypes)
# contractors = payments_dd['contractor_id']
# contractor_ids = payments_dd['contractor_id'].value_counts().head(5000).index
# payments_dd['contractor_id'] = payments_dd['contractor_id'].mask(~payments_dd['contractor_id'].isin(contractor_ids), '0')
# payments_dd = payments_dd.categorize()
# payments_dd =  dd.get_dummies(payments_dd, columns=['contractor_id'])
# payments_dd = dd.get_dummies(payments_dd, columns=['channel'])
# payments_dd['contractor_id'] = contractors

# dict_features = {}
# for i in contractor_ids.values:
#     dict_features[f'contractor_id_{i}'] = 'sum'
# features1 = {'channel_app': 'sum', 'channel_atm': 'sum', 'channel_pos': 'sum','channel_web': 'sum',\
#                                         'dt_hour': ['mean', 'std'], 'dt_day': ['min', 'max', 'mean', 'std'], 'is_outgoing': ['sum', 'count']}

# features1.update(dict_features)
# df = payments_dd.groupby('client_id').agg(features1)
# df.columns = ['_'.join(col).rstrip('_') for col in df.columns.values]
# percentage_outgoing = dd.Aggregation('percentage_outgoing', 
#                               lambda x: x.agg(percentage_outgoing), 
#                               lambda y: y.agg(percentage_outgoing)
#                               )

In [39]:
def generate_features(payments):
    contractors = payments['contractor_id']
    contractor_ids = payments['contractor_id'].value_counts().head(100).index
    payments.loc[~payments['contractor_id'].isin(contractor_ids), 'contractor_id']='-1'
    
    payments = pd.get_dummies(payments, columns=['contractor_id'])
    payments = pd.get_dummies(payments, columns=['channel'])
    payments['contractor_id'] = contractors
    
    dict_features = {}
    for i in contractor_ids.values:
        dict_features[f'contractor_id_{i}'] = 'sum'
        
#     dict_features = {}
#     for i in range(len(contractor_ids)):
#         dict_features[f'{i}'] = 'sum'    
        
    features1 = {'channel_app': 'sum', 'channel_atm': 'sum', 'channel_pos': 'sum','channel_web': 'sum',\
                                        'dt_hour': ['skew', 'mean', 'std'], 'dt_day': ['min', 'max', 'skew', 'mean', 'std', 'nunique'], 'is_outgoing': ['sum', 'count'], \
                                        'contractor_id':'nunique', 'cos': 'mean', 'cos_h': 'mean', 'sin': 'mean', 'sin_h': 'mean'}
    features1.update(dict_features)
    
    df = payments.groupby('client_id', as_index=False, sort=False).agg(features1)
    df.columns = ['_'.join(col).rstrip('_') for col in df.columns.values]
    df = df.set_index('client_id')
    
    
    groupby_client = payments.groupby('client_id')
    
    fts = groupby_client.agg({'amount':['mean', 'std', 'min', 'max', 'sum']})
    fts[[f'flag_{i}_count' for i in range(12)]] = groupby_client[[f'flag_{i}' for i in range(12)]].sum()
    
    is_outgoing_counts = groupby_client['is_outgoing'].apply(lambda x: percentage_outgoing(x))
    fts['0% outgoing'] = is_outgoing_counts
    
    fts = fts.merge(df, left_index = True, right_index=True)
    
#     fts['flags_sum'] = groupby_client.apply(lambda x: sum(x.loc[:, 'flag_0':'flag_11'].sum()))

#     most_channel = groupby_client.apply(lambda x: x['channel'].value_counts().index[0])
#     fts['most channel'] = most_channel

    return fts

In [40]:
payments1 = payments.copy()
features = generate_features(payments1)
features

<ipython-input-39-6d61b7407042>:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = payments.groupby('client_id', as_index=False, sort=False).agg(features1)
<ipython-input-39-6d61b7407042>:45: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  fts = fts.merge(df, left_index = True, right_index=True)


,"(amount, mean)","(amount, std)","(amount, min)","(amount, max)","(amount, sum)","(flag_0_count, )","(flag_1_count, )","(flag_2_count, )","(flag_3_count, )","(flag_4_count, )",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
client_id,,,,,,,,,,,,,,,,,,,,,
100045,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100055,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100068,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100076,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100089,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99975,5.391718e+07,1.590713e+08,4660,859557956,2695859085,20,0,1,19,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
999759,2.389214e+08,1.122544e+09,4742,6612336710,65225539048,158,0,0,13,3,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
999784,3.565689e+08,7.886591e+08,28594,3594312408,30308358385,0,0,0,24,12,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0


In [45]:
i = 0
features.to_csv('features_data2_{}.csv'.format(i))

In [46]:
features = pd.read_csv('features_data2_{}.csv'.format(0))
features = features.set_index('client_id')

In [47]:
features

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
client_id,,,,,,,,,,,,,,,,,,,,,
100045,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100055,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100068,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100076,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
100089,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99975,5.391718e+07,1.590713e+08,4660,859557956,2695859085,20,0,1,19,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
999759,2.389214e+08,1.122544e+09,4742,6612336710,65225539048,158,0,0,13,3,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
999784,3.565689e+08,7.886591e+08,28594,3594312408,30308358385,0,0,0,24,12,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0


In [57]:
def generate_features_contractors(payments, i):
    payments1 = payments.copy()
    contractor_ids = payments1['contractor_id'].value_counts().iloc[i*200:(i+1)*200].index
    payments1.loc[~payments1['contractor_id'].isin(contractor_ids), 'contractor_id']='-1'
    payments1 = pd.get_dummies(payments1, columns=['contractor_id'])


    dict_features = {}
    for i in contractor_ids.values:
        dict_features[f'contractor_id_{i}'] = 'sum'

    
    df = payments1.groupby('client_id', as_index=False, sort=False).agg(dict_features)
#     df.columns = ['_'.join(col).rstrip('_') for col in df.columns.values]
    
#     df = df.set_index('client_id')

    return df

In [49]:
i = 1
payments1 = payments.copy()
contractor1 = generate_features_contractors(payments1, i)
contractor1.head()

,client_id,contractor_id_63809,contractor_id_314433,contractor_id_385552,contractor_id_913643,contractor_id_809737,contractor_id_229481,contractor_id_963007,contractor_id_685372,contractor_id_815674,...,contractor_id_827000,contractor_id_89794,contractor_id_922094,contractor_id_537254,contractor_id_608193,contractor_id_936940,contractor_id_464551,contractor_id_118625,contractor_id_810414,contractor_id_768140
0,100045,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
1,100055,0.0,0,0.0,0.0,0.0,0.0,72.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
2,100068,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
3,100076,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
4,100089,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0


In [50]:
contractor1.shape

(67005, 301)

In [51]:
contractor1 = contractor1.set_index('client_id') #set index
contractor1.index = contractor1.index.astype(int) #change index type
contractor1 = contractor1.reindex(features.index) #sort in order by feature dataframe

In [52]:
contractor1.to_csv('features_data2_{}.csv'.format(i), index=False)

In [90]:
contractor1 = pd.read_csv('features_data2_{}.csv'.format(1))

In [55]:
contractor1.head()

,contractor_id_63809,contractor_id_314433,contractor_id_385552,contractor_id_913643,contractor_id_809737,contractor_id_229481,contractor_id_963007,contractor_id_685372,contractor_id_815674,contractor_id_792219,...,contractor_id_827000,contractor_id_89794,contractor_id_922094,contractor_id_537254,contractor_id_608193,contractor_id_936940,contractor_id_464551,contractor_id_118625,contractor_id_810414,contractor_id_768140
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
1,0.0,0,0.0,0.0,0.0,0.0,72.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
2,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
3,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0
4,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0,0,0,0.0,0.0,0,0


In [58]:
for i in range(2, 4):
    payments1 = payments.copy()
    contractor1 = generate_features_contractors(payments1, i)
    
    contractor1 = contractor1.set_index('client_id') #set index
    contractor1.index = contractor1.index.astype(int) #change index type
    contractor1 = contractor1.reindex(features.index) #sort in order by feature dataframe
    
    contractor1.to_csv('features_data2_{}.csv'.format(i), index=False)

In [59]:
features1 = features.reset_index().iloc[:,1:]

In [60]:
features1

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
0,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
1,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
2,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
3,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
4,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67000,5.391718e+07,1.590713e+08,4660,859557956,2695859085,20,0,1,19,5,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
67001,2.389214e+08,1.122544e+09,4742,6612336710,65225539048,158,0,0,13,3,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
67002,3.565689e+08,7.886591e+08,28594,3594312408,30308358385,0,0,0,24,12,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0
67003,1.119933e+07,1.871622e+07,24390,103498839,828750092,37,0,4,9,8,...,0.0,0.0,0.0,0,0.0,0.0,0,0,0.0,0.0


In [61]:
for i in range(1, 4):
    contractor1 = pd.read_csv('features_data2_{}.csv'.format(i))
    
    features1 = pd.concat([features1, contractor1], axis=1)

In [62]:
features1.head()

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
0,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [63]:
features1.to_csv('Final_Features_Dataframe2.csv', index=False)

In [64]:
pd.read_csv('Final_Features_Dataframe2.csv')

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
0,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67000,5.391718e+07,1.590713e+08,4660,859557956,2695859085,20,0,1,19,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
67001,2.389214e+08,1.122544e+09,4742,6612336710,65225539048,158,0,0,13,3,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
67002,3.565689e+08,7.886591e+08,28594,3594312408,30308358385,0,0,0,24,12,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
67003,1.119933e+07,1.871622e+07,24390,103498839,828750092,37,0,4,9,8,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [65]:
features1.shape

(67005, 838)

In [66]:
import random
SEED = 0xCAFEC0DE

random.seed(SEED)
np.random.seed(SEED)

In [67]:
features1.index = features.index

In [68]:
features1.index = features1.index.astype(str)

In [69]:
features1.head()

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
client_id,,,,,,,,,,,,,,,,,,,,,
100045,8.362072e+06,1.733687e+07,22,124737631,3988708134,203,0,60,47,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
100055,2.201262e+07,6.029931e+07,2238,766121312,14330218840,206,0,42,151,68,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
100068,3.782283e+07,8.493012e+07,34043,681967564,19630047635,394,0,22,8,18,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
100076,1.155591e+07,3.589745e+07,29,255864840,1074700013,63,0,11,8,10,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
100089,3.065695e+07,4.485693e+07,1519,645257028,48560605079,264,0,52,112,5,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [70]:
def stratified_split_cached(X, y, split_idx_file):
    if os.path.isfile(split_idx_file):
        with open(split_idx_file, 'r') as f:
            split_json = json.load(f)
        train_idx, val_idx = split_json['train'], split_json['val']
    else:
        y_shuffle = sku.shuffle(y, random_state=SEED)
        train_idx, _, val_idx, _ = iterative_train_test_split(np.expand_dims(y_shuffle.index, 1), np.array(y_shuffle), test_size=0)
        train_idx, val_idx = train_idx.squeeze(1), val_idx.squeeze(1)
        with open(split_idx_file, 'w') as f:
            json.dump({'train': list(train_idx), 'val': list(val_idx)}, f)
    return X.loc[train_idx], y.loc[train_idx], X.loc[val_idx], y.loc[val_idx]

In [75]:
X_train, y_train, X_val, y_val = stratified_split_cached(features1, target, 'split_cache5.json')

In [76]:
len(X_train), len(y_train), len(X_val), len(y_val)

(67005, 67005, 0, 0)

In [77]:
X_train

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
client_id,,,,,,,,,,,,,,,,,,,,,
469347,3.544950e+07,7.754265e+07,12380,1098312208,18362842881,57,0,35,59,32,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
31053,8.018193e+07,1.162911e+08,73,594908243,7136192030,0,0,0,9,9,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
726736,1.782003e+08,1.873942e+08,31134,726200213,13899625770,0,0,0,28,10,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
438388,2.085516e+07,3.834703e+07,2409,170421023,3837349790,97,0,21,13,24,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
592788,5.818194e+07,6.463688e+07,238581,314723245,9716383639,15,0,3,8,12,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399347,3.668457e+07,6.428507e+07,63384,357004808,5686108625,22,0,21,41,10,...,2.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
470557,4.295133e+07,8.892837e+07,2928,506319917,4080376533,2,0,0,43,0,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
42811,5.099055e+07,1.492271e+08,88,991314308,19733341173,151,41,17,51,11,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [78]:
y_train

,type_0,type_1,type_2,type_3,type_4,type_5,type_6,type_7,type_8,type_9,...,type_25,type_26,type_27,type_28,type_29,type_30,type_31,type_32,type_33,type_34
client_id,,,,,,,,,,,,,,,,,,,,,
469347,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
31053,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
726736,1,0,1,0,1,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
438388,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,0,0,1
592788,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399347,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
470557,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
42811,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
X_train.to_csv('X_train_final_2.csv')

In [80]:
y_train.to_csv('y_train_final_2.csv')

## Modelling

### XGBoost

In [ ]:
model = XGBClassifier(eval_metric=fbeta_score, max_depth=8, n_estimators=5000, tree_method='gpu_hist', predictor = 'gpu_predictor')

In [ ]:
model.fit(X_train.iloc[:-300], y_train)

In [ ]:
prediction = model.predict(X_val.iloc[:-300])

In [ ]:
prediction

In [ ]:
print(fbeta_score(y_val, prediction, beta=0.5, average='micro', zero_division=0))

In [ ]:
print(classification_report(y_val, prediction))

In [ ]:
feature_result = pd.DataFrame(model.feature_importances_, X_train.iloc[:-300].columns)
feature_result.columns = ['result']
feature_result = feature_result.sort_values(by='result', ascending=False)
feature_result = feature_result[feature_result['result']>0.001]

In [ ]:
plt.subplots(figsize=(10, 8), dpi=100)
plt.barh(feature_result.index, feature_result['result'])

plt.title('Значимость признаков', fontsize=16)
plt.xlabel('Значение', fontsize=14)
plt.ylabel('Признаки', fontsize=14)
plt.show()

In [ ]:
from sklearn.model_selection import GridSearchCV

allow_optimize = 1
if allow_optimize:
    param_grid={'max_depth': [4,5,6],
            'n_estimators': [300, 500, 700, 800, 1000],
            'min_child_weight' : [1,2,3,4],
            'gpu_id' : [0]
        }

    classifier = XGBClassifier(tree_method = 'gpu_hist', predictor = 'gpu_predictor')
    CV_classifier = GridSearchCV(classifier, param_grid, cv=2, scoring="accuracy", n_jobs= -1, return_train_score = True, verbose = 3)
    CV_classifier.fit(X_train.iloc[:-300], y_train)
    
    print("The best hyperparameters are : ","\n")
    print(CV_classifier.best_params_)

In [ ]:
if allow_optimize: 
    CV_regressor = CV_regressor.best_estimator_
else:
    CV_regressor = XGBRegressor(tree_method = 'gpu_hist', predictor = 'gpu_predictor', gpu_id = 0, max_depth = 4, n_estimators = 100)
CV_regressor.fit(X, y)

## Submission

In [107]:
PAYMENTS_TEST_PATH

'data/data/payments_test.csv'

In [10]:
payments_test = pd.read_csv(PAYMENTS_TEST_PATH, dtype=payments_dtypes)
payments_test

,client_id,contractor_id,is_outgoing,amount,dt_day,dt_hour,channel,flag_0,flag_1,flag_2,flag_3,flag_4,flag_5,flag_6,flag_7,flag_8,flag_9,flag_10,flag_11
0,303546,NaN,True,378449,50,2,NaN,False,False,False,True,False,False,False,False,False,False,False,False
1,59719,NaN,True,281527,267,2,pos,True,False,False,False,False,False,False,False,False,False,False,False
2,25428,90165,False,2105146744,294,9,NaN,False,False,False,False,False,False,False,False,False,False,False,False
3,766314,NaN,True,2156082,207,14,pos,True,False,False,False,False,False,False,False,False,False,False,False
4,465049,NaN,True,51694546,211,3,atm,True,False,True,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8553957,794745,773056,False,858062290,198,16,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553958,362168,904134,False,8540123,130,13,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553959,500006,471487,False,65346645,196,12,NaN,False,False,False,False,False,False,False,False,False,False,False,False
8553960,653741,NaN,True,3296030,43,0,NaN,True,False,False,False,False,False,False,False,False,False,False,False


In [11]:
sort_payments_test = payments_test.sort_values(by=['client_id', 'dt_day', 'dt_hour'])

In [12]:
sort_payments_test['contractor_id'] = sort_payments_test['contractor_id'].fillna(method='bfill')

In [15]:
sort_payments_test['contractor_id'] = sort_payments_test['contractor_id'].fillna(method='ffill')

In [17]:
payments_test = sort_payments_test

In [18]:
payments_test['cos'] = np.cos(payments_test['dt_day'])
payments_test['cos_h'] = np.cos(payments_test['dt_hour'])
payments_test['sin'] = np.sin(payments_test['dt_day'])
payments_test['sin_h'] = np.sin(payments_test['dt_hour'])

In [25]:
def generate_features(payments, payments_contractors):
    payments = payments.copy()
    contractors = payments['contractor_id']
    contractor_ids = payments_contractors['contractor_id'].value_counts().head(100).index
    payments.loc[~payments['contractor_id'].isin(contractor_ids), 'contractor_id']='-1'
    
    payments = pd.get_dummies(payments, columns=['contractor_id'])
    payments = pd.get_dummies(payments, columns=['channel'])
    payments['contractor_id'] = contractors
    
    dict_features = {}
    for i in contractor_ids.values:
        dict_features[f'contractor_id_{i}'] = 'sum'
 
        
    features1 = {'channel_app': 'sum', 'channel_atm': 'sum', 'channel_pos': 'sum','channel_web': 'sum',\
                                        'dt_hour': ['skew', 'mean', 'std'], 'dt_day': ['min', 'max', 'skew', 'mean', 'std', 'nunique'], 'is_outgoing': ['sum', 'count'], \
                                        'contractor_id':'nunique', 'cos': 'mean', 'cos_h': 'mean', 'sin': 'mean', 'sin_h': 'mean'}
    features1.update(dict_features)
    
    df = payments.groupby('client_id', as_index=False, sort=False).agg(features1)
    df.columns = ['_'.join(col).rstrip('_') for col in df.columns.values]
    df = df.set_index('client_id')
    
    
    groupby_client = payments.groupby('client_id')
    
    fts = groupby_client.agg({'amount':['mean', 'std', 'min', 'max', 'sum']})
    fts[[f'flag_{i}_count' for i in range(12)]] = groupby_client[[f'flag_{i}' for i in range(12)]].sum()
    
    is_outgoing_counts = groupby_client['is_outgoing'].apply(lambda x: percentage_outgoing(x))
    fts['0% outgoing'] = is_outgoing_counts
    
    fts = fts.merge(df, left_index = True, right_index=True)
    
#     fts['flags_sum'] = groupby_client.apply(lambda x: sum(x.loc[:, 'flag_0':'flag_11'].sum()))

#     most_channel = groupby_client.apply(lambda x: x['channel'].value_counts().index[0])
#     fts['most channel'] = most_channel

    return fts

In [111]:
payments_test1 = payments_test.copy()
features_test = generate_features(payments_test1, payments)
features_test

<ipython-input-110-b445e30c137a>:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df = payments.groupby('client_id', as_index=False, sort=False).agg(features1)
<ipython-input-110-b445e30c137a>:46: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  fts = fts.merge(df, left_index = True, right_index=True)


,"(amount, mean)","(amount, std)","(amount, min)","(amount, max)","(amount, sum)","(flag_0_count, )","(flag_1_count, )","(flag_2_count, )","(flag_3_count, )","(flag_4_count, )",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
client_id,,,,,,,,,,,,,,,,,,,,,
100050,3.536920e+07,7.702827e+07,798,474601237,5659071739,7,0,0,35,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100128,2.074924e+08,5.213882e+08,2401,4263233615,20126762544,0,0,0,19,11,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100159,1.516675e+08,2.081792e+08,17615,1348514255,113143937688,0,0,0,11,52,...,0,0.0,0.0,0,0,0.0,0,0,0,0
10018,5.424148e+07,1.042072e+08,43,808270306,18984516728,0,0,0,17,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100237,2.005865e+08,3.939886e+08,31511,2521806684,21262166365,16,0,12,28,2,...,0,0.0,0.0,0,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999572,4.462335e+07,8.773594e+07,2137,1586323663,34359979305,0,0,0,22,76,...,0,74.0,0.0,0,0,0.0,0,0,0,0
99966,5.238860e+07,2.225828e+08,578,2924402093,44949423064,701,0,14,17,26,...,0,0.0,0.0,0,0,0.0,0,0,0,0
999662,3.545021e+07,9.601183e+07,19606,947093072,14180084442,232,0,20,8,20,...,0,0.0,0.0,0,0,0.0,0,0,0,0


In [112]:
i = 0
features_test.to_csv('features_test_data2_{}.csv'.format(i))

In [26]:
features_test = pd.read_csv('features_test_data2_{}.csv'.format(0))
features_test = features_test.set_index('client_id')

In [27]:
features_test.head()

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
client_id,,,,,,,,,,,,,,,,,,,,,
100050,3.536920e+07,7.702827e+07,798,474601237,5659071739,7,0,0,35,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100128,2.074924e+08,5.213882e+08,2401,4263233615,20126762544,0,0,0,19,11,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100159,1.516675e+08,2.081792e+08,17615,1348514255,113143937688,0,0,0,11,52,...,0,0.0,0.0,0,0,0.0,0,0,0,0
10018,5.424148e+07,1.042072e+08,43,808270306,18984516728,0,0,0,17,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
100237,2.005865e+08,3.939886e+08,31511,2521806684,21262166365,16,0,12,28,2,...,0,0.0,0.0,0,0,0.0,0,0,0,0


In [ ]:
# 100, 400
# 400, 600

In [93]:
def generate_features_contractors(payments1, payments_contractors, i):
    payments1 = payments1.copy()
    contractor_ids = payments_contractors['contractor_id'].value_counts().iloc[i*200:(i+1)*200].index
    payments1.loc[~payments1['contractor_id'].isin(contractor_ids), 'contractor_id']='-1'
    
    
    add_inx = np.array(list(set(contractor_ids.values) - set(payments1['contractor_id'].values)))
    print(add_inx)
    
    payments1 = pd.get_dummies(payments1, columns=['contractor_id'])
    
    for i in add_inx:
        payments1[f'contractor_id_{i}'] = 0

    dict_features = {}
    for i in contractor_ids.values:
        dict_features[f'contractor_id_{i}'] = 'sum'

    
    df = payments1.groupby('client_id', as_index=False, sort=False).agg(dict_features)
#     df.columns = ['_'.join(col).rstrip('_') for col in df.columns.values]
    
#     df = df.set_index('client_id')

    return df

In [84]:
i = 1
payments_test1 = payments_test.copy()
contractor1_test = generate_features_contractors(payments_test1, payments, i)
contractor1_test.head()

['687179']


,client_id,contractor_id_63809,contractor_id_314433,contractor_id_385552,contractor_id_913643,contractor_id_809737,contractor_id_229481,contractor_id_963007,contractor_id_685372,contractor_id_815674,...,contractor_id_827000,contractor_id_89794,contractor_id_922094,contractor_id_537254,contractor_id_608193,contractor_id_936940,contractor_id_464551,contractor_id_118625,contractor_id_810414,contractor_id_768140
0,100050,0,0.0,0,0,0,18.0,0,0,0,...,0,0,0,0.0,0.0,0,0.0,0,0,0
1,100128,0,0.0,0,0,0,0.0,0,0,0,...,0,0,0,0.0,0.0,0,0.0,0,0,0
2,100159,0,0.0,0,0,0,0.0,0,0,0,...,0,0,0,0.0,0.0,0,0.0,0,0,0
3,10018,0,0.0,0,0,0,0.0,0,0,0,...,0,0,0,0.0,0.0,0,0.0,0,0,0
4,100237,0,0.0,0,0,0,0.0,0,0,0,...,0,0,0,0.0,0.0,0,0.0,0,0,0


In [85]:
contractor1_test = contractor1_test.set_index('client_id') #set index
contractor1_test.index = contractor1_test.index.astype(int) #change index type
contractor1_test = contractor1_test.reindex(features_test.index) #sort in order by feature dataframe

In [86]:
contractor1_test.to_csv('features_test_data2_{}.csv'.format(i), index=False)

In [87]:
contractor1_test = pd.read_csv('features_test_data2_{}.csv'.format(1))

In [95]:
for i in range(2, 4):
    print(i)
    payments_test1 = payments_test.copy()
    contractor1_test = generate_features_contractors(payments_test1, payments, i)
    
    contractor1_test = contractor1_test.set_index('client_id') #set index
    contractor1_test.index = contractor1_test.index.astype(int) #change index type
    contractor1_test = contractor1_test.reindex(features_test.index) #sort in order by feature dataframe
    
    contractor1_test.to_csv('features_test_data2_{}.csv'.format(i), index=False)
    

2
['897573' '618298' '477947' '733106' '311188' '417778']
3
['860273' '437783' '728061' '633496' '944266' '871271' '829399' '796283'
 '49583' '529303' '913021' '926739' '241715' '2832' '876559' '951217'
 '999291' '764311' '833511' '807059' '296193' '408629' '683419' '430630'
 '124620' '971781' '682793' '16219' '953119' '519921' '419202' '552529'
 '688164' '575159' '25404' '113528' '317285' '96923' '967165' '128322'
 '830556' '68312' '579970' '852430' '846092' '849355' '137628' '824071'
 '225337' '932980' '185059' '770279' '300579' '196712' '384719' '23116'
 '769926' '167711' '152140']


In [96]:
features1_test = features_test.reset_index().iloc[:,1:]

In [97]:
features1_test

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_390377_sum,contractor_id_767214_sum,contractor_id_603437_sum,contractor_id_295580_sum,contractor_id_462711_sum,contractor_id_351749_sum,contractor_id_583567_sum,contractor_id_590169_sum,contractor_id_15702_sum,contractor_id_32285_sum
0,3.536920e+07,7.702827e+07,798,474601237,5659071739,7,0,0,35,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
1,2.074924e+08,5.213882e+08,2401,4263233615,20126762544,0,0,0,19,11,...,0,0.0,0.0,0,0,0.0,0,0,0,0
2,1.516675e+08,2.081792e+08,17615,1348514255,113143937688,0,0,0,11,52,...,0,0.0,0.0,0,0,0.0,0,0,0,0
3,5.424148e+07,1.042072e+08,43,808270306,18984516728,0,0,0,17,13,...,0,0.0,0.0,0,0,0.0,0,0,0,0
4,2.005865e+08,3.939886e+08,31511,2521806684,21262166365,16,0,12,28,2,...,0,0.0,0.0,0,0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22040,4.462335e+07,8.773594e+07,2137,1586323663,34359979305,0,0,0,22,76,...,0,74.0,0.0,0,0,0.0,0,0,0,0
22041,5.238860e+07,2.225828e+08,578,2924402093,44949423064,701,0,14,17,26,...,0,0.0,0.0,0,0,0.0,0,0,0,0
22042,3.545021e+07,9.601183e+07,19606,947093072,14180084442,232,0,20,8,20,...,0,0.0,0.0,0,0,0.0,0,0,0,0
22043,1.040799e+08,2.235370e+08,31020,2255212467,29454608233,138,11,14,22,15,...,0,0.0,0.0,0,0,0.0,0,0,0,0


In [16]:
del payments

In [98]:
for i in range(1, 4):
    contractor1_test = pd.read_csv('features_test_data2_{}.csv'.format(i))
    
    features1_test = pd.concat([features1_test, contractor1_test], axis=1)

In [99]:
features1_test.head()

,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')","('flag_4_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
0,3.536920e+07,7.702827e+07,798,474601237,5659071739,7,0,0,35,13,...,0,0,0,0,0,0,0,0,0,0
1,2.074924e+08,5.213882e+08,2401,4263233615,20126762544,0,0,0,19,11,...,0,0,0,0,0,0,0,0,0,0
2,1.516675e+08,2.081792e+08,17615,1348514255,113143937688,0,0,0,11,52,...,0,0,0,0,0,0,0,0,0,0
3,5.424148e+07,1.042072e+08,43,808270306,18984516728,0,0,0,17,13,...,0,0,0,0,0,0,0,0,0,0
4,2.005865e+08,3.939886e+08,31511,2521806684,21262166365,16,0,12,28,2,...,0,0,0,0,0,0,0,0,0,0


In [100]:
(pd.read_csv('Final_Features_Dataframe2.csv').columns.values == features1_test.columns.values).any()

True

In [101]:
features1_test.index = features_test.index

In [102]:
features1_test.to_csv('Submission_Features_Dataframe2.csv')

In [103]:
pd.read_csv('Submission_Features_Dataframe2.csv')

,client_id,"('amount', 'mean')","('amount', 'std')","('amount', 'min')","('amount', 'max')","('amount', 'sum')","('flag_0_count', '')","('flag_1_count', '')","('flag_2_count', '')","('flag_3_count', '')",...,contractor_id_986425,contractor_id_3425,contractor_id_137239,contractor_id_373618,contractor_id_26746,contractor_id_860273,contractor_id_96923,contractor_id_853363,contractor_id_953119,contractor_id_124834
0,100050,3.536920e+07,7.702827e+07,798,474601237,5659071739,7,0,0,35,...,0,0,0,0,0,0,0,0,0,0
1,100128,2.074924e+08,5.213882e+08,2401,4263233615,20126762544,0,0,0,19,...,0,0,0,0,0,0,0,0,0,0
2,100159,1.516675e+08,2.081792e+08,17615,1348514255,113143937688,0,0,0,11,...,0,0,0,0,0,0,0,0,0,0
3,10018,5.424148e+07,1.042072e+08,43,808270306,18984516728,0,0,0,17,...,0,0,0,0,0,0,0,0,0,0
4,100237,2.005865e+08,3.939886e+08,31511,2521806684,21262166365,16,0,12,28,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22040,999572,4.462335e+07,8.773594e+07,2137,1586323663,34359979305,0,0,0,22,...,0,0,0,0,0,0,0,0,0,0
22041,99966,5.238860e+07,2.225828e+08,578,2924402093,44949423064,701,0,14,17,...,0,0,0,0,0,0,0,0,0,0
22042,999662,3.545021e+07,9.601183e+07,19606,947093072,14180084442,232,0,20,8,...,0,0,0,0,0,0,0,0,0,0
22043,999674,1.040799e+08,2.235370e+08,31020,2255212467,29454608233,138,11,14,22,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
prediction_test = model.predict(features1_test.iloc[:-300])

In [ ]:
prediction_test

In [ ]:
classes = []
for i in range(35):
    classes.append(f'type_{i}')

In [ ]:
submission = pd.DataFrame(prediction_test, columns = classes)
submission.head()

In [ ]:
submission['client_id'] = features_test.index

In [ ]:
classes.insert(0, 'client_id')

In [ ]:
submission = submission[classes]

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index=False)